In [1]:
%use dataframe, kandy

In [2]:
USE {
    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [3]:
fun DataFrame<*>.addProjectColumn(): DataFrame<*> =
    add(this["benchmark"].map { (it as String).substringBefore('-') }.rename("project"))

val folder = "../output/base"

val smtData = DataFrame.readCSV("${folder}/smtData.csv").addProjectColumn()
val fullSmtData = DataFrame.read("${folder}/fullSmtData.csv").addProjectColumn()

println("smtData: ${smtData.columnNames()}")
println("fullSmtData: ${fullSmtData.columnNames()}")


smtData: [benchmark, cnt, sat, unsat, unknown, project]
fullSmtData: [benchmark, file, result, numberOfVariables, reps, numberOfClauses, height, ucNumberOfVariables, ucReps, ucNumberOfClauses, ucHeight, project]


In [4]:
import org.jetbrains.letsPlot.scale.guideColorbar
import org.jetbrains.letsPlot.scale.guideLegend

smtData.plot {
    points {
        x(sat) { axis.name = "SAT" }
        y(unsat) { axis.name = "UNSAT" }
        color(sat.values.zip(unsat.values).map { (s, u) -> s < u }) {
            scale = categorical(
                false to Color.GREEN, true to Color.RED
            )
            legend {
                breaksLabeled(true to "SAT < UNSAT", false to "UNSAT > SAT")
            }
        }
    }
    abLine {
        slope.constant(1)
        intercept.constant(0)
        color = Color.ORANGE
        type = LineType.DASHED
    }
    layout {
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="uFA5iJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"color":[false,false,true,false,false,false,true,true,true,false,true,false,false,false,false,true,true,false,false,true,false,true,true,false,true,false,false,false,true,false,false,false,false,true,true,false,true,true,false,false,true,false,false,true,true,true,false,true,false,true,false,false,false,false],
"sat":[77.0,252.0,5.0,6.0,94.0,3.0,78.0,19.0,0.0,33.0,16.0,82.0,196.0,107.0,25.0,41.0,2.0,64.0,45.0,30.0,131.0,34.0,16.0,45.0,114.0,50.0,54.0,237.0,44.0,97.0,84.0,122.0,88.0,49.0,31.0,9.0,14.0,7.0,1.0,15.0,6.0,60.0,9.0,5.0,122.0,12.0,17.0,57.0,20.0,12.0,11.0,6.0,18.0,60.0],
"unsat":[21.0,73.0,6.0,2.0,40.0,2.0,185.0,50.0,98.0,8.0,17.0,75.0,50.0,2.0,7.0,53.0,111.0,15.0,40.0,45.0,63.0,309.0,24.0,2.0,124.0,9.0,1.0,236.0,72.0,14.0,47.0,83.0,60.0,69.0,45.0,8.0,22.0,11.0,0.0,8.0,11.0,10.0,0.0,12.0,1037.0,56.0,2.0,67.0,3.0,16.0,1.0,5.0,0.0,8.0]
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"SAT",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"UNSAT",
"limits":[null,null]
},{
"aesthetic":"color",
"breaks":[true,false],
"values":["#3ba272","#ee6666"],
"limits":[false,true],
"labels":["SAT < UNSAT","UNSAT > SAT"]
}],
"layers":[{
"mapping":{
"x":"sat",
"y":"unsat",
"color":"color"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"point",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"color":"#fc8452",
"intercept":0.0,
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"abline",
"slope":1.0,
"data":{
}
}]
};
 var plotContainer = document.getElementById("uFA5iJ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 UNSAT 
 
 
 
 
 SAT 
 
 
 
 
 
 
 
 
 color 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 UNSAT > SAT 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 SAT < UNSAT

In [5]:
smtData.plot { 
    val transform: (Number) -> Double = {
        log(it.toDouble() + 1, 5.0)
    }
    val s = sat.map { transform(it) }
    val u = unsat.map { transform(it) }
    
    points { 
        x(s) {
            axis {
                name = "SAT"
                breaksLabeled(0.0 to "0", 1.0 to "4", 2.0 to "24", 3.0 to "124")
            }
        }
        y(u) { 
            axis {
                name = "UNSAT"
                breaksLabeled(0.0 to "0", 1.0 to "4", 2.0 to "24", 3.0 to "124", 4.0 to "624")
            }
        }
        color(sat.values.zip(unsat.values).map { (s, u) -> s < u }) {
            scale = categorical(
                false to Color.GREEN, true to Color.RED
            )
            legend {
                breaksLabeled(true to "SAT < UNSAT", false to "UNSAT > SAT")
            }
        }
    }
    abLine {
        slope.constant(1)
        intercept.constant(0)
        color = Color.ORANGE
        type = LineType.DASHED
    }
    layout {
        size = 800 to 400
    }
//    facetWrap(2) {
//        facet(project)
//    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="t2A6je"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"color":[false,false,true,false,false,false,true,true,true,false,true,false,false,false,false,true,true,false,false,true,false,true,true,false,true,false,false,false,true,false,false,false,false,true,true,false,true,true,false,false,true,false,false,true,true,true,false,true,false,true,false,false,false,false],
"sat":[2.7069753937264607,3.438088195871358,1.1132827525593785,1.2090619551221675,2.8294828004351507,0.8613531161467861,2.7148905954742335,1.8613531161467862,0.0,2.1910509857959815,1.7603744277225881,2.7455800398746546,3.2826390430605152,2.9091716996047423,2.0243691992404753,2.322344707681546,0.6826061944859854,2.5936926411670824,2.378868651539773,2.133656214977323,3.0338554130377493,2.2090619551221677,1.7603744277225881,2.378868651539773,2.9481920934663797,2.4429806222085735,2.489896102404978,3.4001129409181488,2.3652123889719707,2.8488004683177284,2.760374427722588,2.9899782515341085,2.7889465850494126,2.4306765580733933,2.1533827903669653,1.4306765580733933,1.6826061944859854,1.2920296742201791,0.43067655807339306,1.7227062322935722,1.2090619551221675,2.554229543378943,1.4306765580733933,1.1132827525593785,2.9899782515341085,1.5936926411670824,1.7958889470453636,2.522895092240893,1.8916681496081529,1.5936926411670824,1.5439593106327716,1.2090619551221675,1.8294828004351504,2.554229543378943],
"unsat":[1.9205726604783713,2.6742660030263226,1.2090619551221675,0.6826061944859854,2.3073720570481235,0.6826061944859854,3.2469389675367015,2.4429806222085735,2.855108491376949,1.3652123889719707,1.7958889470453636,2.690835916581937,2.4429806222085735,0.6826061944859854,1.2920296742201791,2.478495141531349,2.9317681874157397,1.7227062322935722,2.3073720570481235,2.378868651539773,2.5840593484403582,3.564332773050716,2.0,0.6826061944859854,3.0000000000000004,1.4306765580733933,0.43067655807339306,3.397496789960219,2.665812336096604,1.6826061944859854,2.405312426779558,2.753021265754939,2.554229543378943,2.639738513195561,2.378868651539773,1.3652123889719707,1.9481920934663797,1.5439593106327716,0.0,1.3652123889719707,1.5439593106327716,1.4898961024049782,0.0,1.5936926411670824,4.315202848193254,2.512088994921136,0.6826061944859854,2.6217275438693743,0.8613531161467861,1.7603744277225881,0.43067655807339306,1.1132827525593785,0.0,1.3652123889719707]
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"breaks":[0.0,1.0,2.0,3.0],
"name":"SAT",
"limits":[null,null],
"labels":["0","4","24","124"]
},{
"aesthetic":"y",
"breaks":[0.0,1.0,2.0,3.0,4.0],
"name":"UNSAT",
"limits":[null,null],
"labels":["0","4","24","124","624"]
},{
"aesthetic":"color",
"breaks":[true,false],
"values":["#3ba272","#ee6666"],
"limits":[false,true],
"labels":["SAT < UNSAT","UNSAT > SAT"]
}],
"layers":[{
"mapping":{
"x":"sat",
"y":"unsat",
"color":"color"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"point",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"color":"#fc8452",
"intercept":0.0,
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"abline",
"slope":1.0,
"data":{
}
}]
};
 var plotContainer = document.getElementById("t2A6je");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 
 
 124 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 124 
 
 
 
 
 
 
 624 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [6]:
fun DataFrame<*>.filterUnsat(): DataFrame<*> = filter { get("result") == "UNSAT" }

val unsatSmtData = fullSmtData.filterUnsat()

In [7]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale

fullSmtData.plot {
    boxplot(project, numberOfVariables)
    boxplot(numberOfVariables, xName = "ALL")
    y {
        axis.expand(additive = 0.5)
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
    layout {
        title = "Variables distribution"
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Fz4vYK"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Variables distribution"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"numberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"ALL",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"numberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"expand":[0.0,0.5],
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[41.0,57.0,67.5,175.0],
"min":[0.0,0.0,4.0,0.0],
"max":[334.0,530.0,214.0,617.0],
"upper":[142.0,225.5,136.75,305.0],
"lower":[13.0,22.0,29.5,94.0],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA",

In [8]:
unsatSmtData.plot {
    boxplot(project, ucNumberOfVariables)
    boxplot(ucNumberOfVariables, xName = "ALL")
    y {
        axis.expand(additive = 0.5)
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
    layout {
        title = "Variables distribution in unsat cores"
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="M8K8ZP"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Variables distribution in unsat cores"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"ucNumberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"ALL",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"ucNumberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"expand":[0.0,0.5],
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[5.0,6.0,8.0,4.0],
"min":[0.0,0.0,2.0,4.0],
"max":[14.0,75.0,15.0,4.0],
"upper":[8.0,32.5,10.0,4.0],
"lower":[3.0,3.0,6.0,4.0],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","SEATA","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON

In [9]:
unsatSmtData.plot {
    boxplot(project, ucNumberOfClauses)
    boxplot(ucNumberOfClauses, xName = "ALL")
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
        axis.expand(additive = 0.5)
    }
    layout {
        title = "Clauses distribution in unsat cores"
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="j8u9Ge"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Clauses distribution in unsat cores"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"ucNumberOfClauses",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"ALL",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"ucNumberOfClauses",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"expand":[0.0,0.5],
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[5.0,7.0,8.0,5.0],
"min":[4.0,1.0,3.0,5.0],
"max":[7.0,80.0,10.0,5.0],
"lower":[5.0,4.0,6.0,5.0],
"upper":[6.0,34.5,8.0,5.0],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FA

In [10]:
fullSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(numberOfVariables) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Variables distribution"
            size = 800 to 400
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Z126z8"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Variables distribution"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density"
},
"stat":"identity",
"data":{
"density":[0.18736859999221475,0.4175423035118008,0.5477081046007676,0.6243132958146417,0.6770201289322269,0.7246890438595258,0.7728534519277389,0.8174830144475603,0.8444691235244494,0.8639406212528665,0.8793390318000939,0.8903723368110067,0.8993793498040988,0.9067557017842064,0.9116752237572534,0.9167869191786964,0.9212277471961218,0.9247581485463845,0.9279645257720862,0.9299637479632771,0.9318573885743096,0.9334131302411768,0.9350206718666979,0.9364867222654252,0.9381908910684054,0.9402058933907206,0.9414889260970721,0.9426318402990482,0.9442952040303184,0.9462524493261264,0.9482338664289395,0.9495577244683571,0.950676885455569,0.952117469924643,0.9533416313850658,0.9544016119770323,0.955706696284779,0.9568939947420385,0.9578916318971878,0.9584263091587156,0.9591286020693895,0.9598391957646125,0.9606655343935129,0.9623113180592215,0.9641486342144943,0.9652728101879775,0.9659288985419346,0.966614578052528,0.9674850288679893,0.9683349611536552,0.969138746041298,0.9700821087963253,0.9708453988084822,0.9712382793139098,0.971476455873054,0.971624588639922,0.9718883587477815,0.9722879166406816,0.9727670723947909,0.9730608916037631,0.9733766461300014,0.9736847695640991,0.974150319327512,0.974651436985639,0.9751639821185705,0.9757192223434359,0.9762724937476999,0.9765400395535145,0.9768499811219348,0.9770770863420555,0.9773042221664386,0.9776437463183443,0.9778773772317495,0.977978660510739,0.9784068797037472,0.9788904990878247,0.9790944289865051,0.9795667819323178,0.9797449386924645,0.9799661534731763,0.9802789929801201,0.9804216732281333,0.9806398536806779,0.9808966658641018,0.9810884306761504,0.9813179047943388,0.9814636661827164,0.981659322144922,0.98197902158251,0.9823409636028121,0.9825924022331177,0.9829934787563214,0.9833082529825699,0.9835743513557862,0.9838281495463378,0.9840195952894373,0.9840378162173472,0.9840712769242981,0.9842686759113405,0.9846579301100334,0.9850039669842622,0.9853263355550059,0.985661422629294,0.9860161926562668,0.986175694852231,0.9861961821423874,0.9862959836216159,0.9866458095247901,0.9871331487134443,0.9872621023053529,0.9872661681910649,0.9872849851507849,0.987469039083607,0.9877631397600491,0.9880514645603763,0.9882740260250327,0.988340173285724,0.988342026664654,0.9883446770422694,0.9884304001194081,0.988634442727841,0.9887692020318145,0.9889297861143886,0.9891953696176208,0.9893923485183171,0.9894758471338703,0.9896628684050129,0.9897744181948123,0.9899766073140714,0.9900375924009194,0.9901419550658008,0.9903778174152315,0.990486828678277,0.9904998920852698,0.9906511311085399,0.9910567702750933,0.9913412755870118,0.9916208241673876,0.991767620718982,0.9919657622744207,0.9920310949746467,0.9920332949368822,0.992059703854388,0.9921604605202038,0.9922027143469493,0.9923056924104794,0.992420199179967,0.9924898979968844,0.992498531238896,0.9925691866613304,0.9926883276498931,0.9927916622066755,0.9928766865671809,0.9930306856537348,0.9931045872158054,0.9931215930616805,0.9932149300980003,0.993260834938503,0.9932672355198439,0.993340288629663,0.9934546188460308,0.9935559738954522,0.9936093308034696,0.9937087985416116,0.9937827347600224,0.9940145229328265,0.9943610765210575,0.9945067225044147,0.9946136300932424,0.994645235227817,0.9946459133362183,0.994645914326117,0

In [12]:
unsatSmtData
    .plot {
        statDensity(ucHeight) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Height distribution"
            size = 800 to 400
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7DyNic"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Height distribution"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density"
},
"stat":"identity",
"data":{
"density":[0.013439661505175443,0.01911135437548247,0.019537640695388134,0.019547450998392144,0.019743799922062395,0.021416325427059624,0.02395376555211047,0.024639363782175464,0.02467552729094045,0.025221446265240074,0.041931437377621686,0.13306911225405205,0.22159462056212845,0.23690861736306945,0.23739983329422704,0.23954031349489513,0.28145885047140257,0.42783666426234407,0.5188684726995832,0.5289509277941353,0.5292092041699608,0.5333971845310757,0.5859625919474462,0.7034839026443133,0.750277019267861,0.7535955960857021,0.7537455718920385,0.7586229453085408,0.7978235613357468,0.8539357121557452,0.8682401145257199,0.868890445824497,0.8690422023752609,0.8732754028186528,0.8950590584671263,0.9150228127876401,0.918281203538892,0.918379183923639,0.9187190348959068,0.9249970581477642,0.9456809864487441,0.9578174430261691,0.9590856794792664,0.9591100401464359,0.9591605485120401,0.9597578574386503,0.9610178167948459,0.9614911455121555,0.9615228229994885,0.9615255162207318,0.9616241280745919,0.9623719218451451,0.9633818415273758,0.9636247490600818,0.9636353490441641,0.9636652721420227,0.9644788665502805,0.9684289982220836,0.9718445745502792,0.9723705499966607,0.9723851787182036,0.9724052175556591,0.9727537516853587,0.9738371704167552,0.9744369562349942,0.9744960944482366,0.9744982558172662,0.9745687266934364,0.9753562065374353,0.9769234554560059,0.9774789570012209,0.9775140414316243,0.9775186831427852,0.9776893244337621,0.9789102222986676,0.9804659203100455,0.9808189571838639,0.980833266156863,0.9808392847185764,0.9809914496386066,0.9816884914190568,0.98225715079216,0.9823397726675434,0.9823419804357936,0.9823484518944068,0.9824548622445274,0.9827669494125207,0.9829299603456835,0.982945124484184,0.9829455084948555,0.9829533711398218,0.9830362642587324,0.9831919174326558,0.9832439704973738,0.9832470812177998,0.9832493630539872,0.9833346083070108,0.9839100589392246,0.9846018819498918,0.9847500077407184,0.9847556560062952,0.9847556943636178,0.9847556944100083,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100182,0.9847556944100224,0.9847556944267746,0.9847557065202599,0.9847572613418714,0.9847928620308454,0.9849380347411166,0.9850434637962858,0.9850570997146251,0.9850574138068722,0.9850574150953555,0.9850574150962967,0.9850574150962969,0.9850574150962969,0.9850574150962969,0.9850574150962969,0.9850574150962969,0.9850574150962969,0.9850574150962974,0.9850574150993077,0.9850574183016849,0.9850580250312017,0.9850784973246322,0.9852015204113569,0.9853331809080209,0.9853582751366596,0.9853591562410615,0.9853627156535437,0.9854394992426081,0.9857349165449384,0.9859373359884649,0.9859620371441459,0.9859625739670865,0.9859625760448381,0.9859625760462702,0.9859625760462709,0.9859625760507329,0.9859625830458466,0.9859645360849689,0.9860616487465419,0.9869216310583598,0.9882779190858308,0.988658864813547,0.9886779400818705,0.9886816228580011,0.9887934619707518,0.9894275561376619,0.9900678267911506,0.9901829660226175,0.990186848346083

In [ ]:
fullSmtData
//    .filter { result == "UNSAT" }
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(numberOfClauses) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Clauses distribution"
            size = 800 to 400
        }
    }

In [ ]:
unsatSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(ucNumberOfVariables) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Variables distribution in unsat cores"
            size = 800 to 400
        }
    }

In [ ]:
unsatSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(ucNumberOfClauses) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Clauses distribution in unsat cores"
            size = 800 to 400
        }
    }

In [ ]:
unsatSmtData
    .filter { ucNumberOfVariables < 15 }
    .plot {
        statDensity(ucReps) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            size = 800 to 400
        }
    }

In [ ]:
unsatSmtData
//    .filter { project == "FASTJSON" }
    .plot {
        points {
            x(numberOfClauses)
            y(ucNumberOfVariables)
        }
//    x {
//        scale = PositionalContinuousScale<Double>(
//            null, null, null, Transformation.LOG2
//        )
//    }
    }

In [ ]:
fullSmtData.plot {
    boxplot(result, numberOfClauses)
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
}

In [ ]:
fullSmtData.plot {
    boxplot(result, numberOfVariables)
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
}

In [ ]:
fullSmtData
    .filter { ucNumberOfVariables < 20 }
    .plot {
        boxplot(project, ucNumberOfClauses.mapIndexed { i, n -> numberOfClauses[i].toLong() * n }.rename("value"))
        y {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
    }